# header file

In [71]:
import numpy as np
import pandas as pd
import math

print('done')

done


In [72]:
np.set_printoptions(suppress=True) #prevent numpy exponential 

In [73]:
path = "/media/mint/New Volume/4_semester_IIIT_D/capstone/hmog_dataset/151985/151985_session_"#13/TouchEvent.csv'

In [74]:
# screen grid (splitting the user screen into y_cross * x_cross)
y_cross = 8
x_cross = 6

# getting the screen size

In [75]:
# to get the minimum and maximum dimension of user screen size 

def screen_size():
    xx_min = 100000
    xx_max = -1
    yy_min = 100000
    yy_max = -1
    for i in range(1,25):
        data = pd.read_csv(path+str(i)+"/TouchEvent.csv", header=None)
        # data.head()
        x_min = np.min(data.iloc[:,6])# 6 and 7 th are the x and y axis of csv file
        x_max = np.max(data.iloc[:,6])
        y_min = np.min(data.iloc[:,7])
        y_max = np.max(data.iloc[:,7])
        if(x_min < xx_min):
            xx_min = x_min
        if(y_min < yy_min):
            yy_min = y_min
        if(x_max > xx_max):
            xx_max = x_max
        if(y_max > yy_max):
            yy_max = y_max
    # print(xx_min)
    # print(xx_max)
    # print(yy_min)
    # print(yy_max)

    x_len = xx_max - xx_min
    # print(x_len)
    fact = x_len / x_cross 
    # print(fact)
    x_axis = np.linspace(fact, x_len-fact, x_cross-1)
    print(x_axis)

    y_len = yy_max - yy_min
    # print(y_len)
    fact = y_len / y_cross 
    # print(fact)
    y_axis = np.linspace(fact, y_len-fact, y_cross-1)
    print(y_axis)
    return x_axis, y_axis

In [76]:
x_axis, y_axis = screen_size()

[ 174.17843367  348.35686733  522.535301    696.71373467  870.89216833]
[  231.00453187   462.00906375   693.01359563   924.0181275   1155.02265938
  1386.02719125  1617.03172313]


# POLICY iteration: parameter initialisation

In [77]:

NO_OF_STATES = 48#24#36#80#48#
NO_OF_ACTIONS = 2# 0 means remain in same state, 1 otherwise  
TRANSITION_PROBABILITY= np.zeros((NO_OF_STATES, NO_OF_ACTIONS, NO_OF_STATES))  # transition probability
TRANSITION_PROBABILITY= TRANSITION_PROBABILITY+ 0.00001
print(TRANSITION_PROBABILITY.shape)
REWARD = np.zeros((NO_OF_STATES, NO_OF_ACTIONS, NO_OF_STATES))  # rewards
REWARD =REWARD + 0.00001 # - 10.1
print(REWARD.shape)

POLICY = [0 for s in range(NO_OF_STATES)]
V = np.zeros(NO_OF_STATES)

GAMMA = 0.7

(48, 2, 48)
(48, 2, 48)


# POLICY iteration

In [78]:
def POLICY_iteration():
    flag = True # whether value changed or not

    while flag:
        flag = False

        for s in range(NO_OF_STATES):
            temp = 0
            for s1 in range(NO_OF_STATES):
                temp += (REWARD[s,POLICY[s],s1] + GAMMA*V[s1]) * TRANSITION_PROBABILITY[s,POLICY[s],s1]   
            V[s] = temp
                
        for s in range(NO_OF_STATES):
            cur = V[s]

            for a in range(NO_OF_ACTIONS):
                temp = 0
                for s1 in range(NO_OF_STATES):
                    temp += (REWARD[s, a, s1] + GAMMA * V[s1]) * TRANSITION_PROBABILITY[s, a, s1] 
                
                if temp > cur:
                    flag = True
                    POLICY[s] = a
                    cur = temp


    print "POLICY:  "
    policy_temp = np.zeros((y_cross,x_cross))
    v_temp = np.zeros((y_cross,x_cross))
    for i in range(y_cross):
        for j in range(x_cross):
            #print(i,j), 
#             print(i*x_cross+j),
            policy_temp[i,j] = POLICY[i*x_cross+j]
            v_temp[i,j] = V[i*x_cross+j]

    print policy_temp
    print v_temp

# Initially data is in milisecond
    
    so milisecond must be combined to form some meaningful things
    
# So, Milisecond data is combined into .1 seconds

In [79]:
def user_data_generation(start_user_id, end_user_id):
    user_movement_data = []

    count = 0
    for k in range(start_user_id, end_user_id+1):
        data = pd.read_csv(path+str(k)+"/TouchEvent.csv", header=None)
        #count = 0
        i = 0
        #for i in range(len(data)-1):
        while i < len(data): 
            #count += 1# total number of rows
            x_cor = []
            y_cor = []
            total_pressure = 0

            time1 = data.iloc[i,1]

            j = i
            while j < len(data):

                time2 = data.iloc[j,1]

                if time2-time1<=100: # .1 seconds session is combined
                    x = data.iloc[j,6]
                    y = data.iloc[j,7]
                    pressure = data.iloc[j,8]

                    x_cor.append(x)
                    y_cor.append(y)
                    total_pressure += pressure    

                    j+= 1

                else:
                    break

            x = np.mean(x_cor)
            y = np.mean(y_cor)

            user_movement_data.append([x,y,total_pressure])

            count = count +1
            i = j

    print(count)

    user_movement_data = pd.DataFrame(user_movement_data)

    return user_movement_data
                 

# Initial 20 session for training

# Last 4 session for testing

In [80]:
user_movement_data = user_data_generation(1,24)
print(user_movement_data.head())
user_movement_data_test = user_data_generation(22,22)
print(user_movement_data_test.head())

13499
            0          1    2
0  922.000000  1357.0000  5.0
1  921.166667  1344.9132  6.0
2  921.000000  1327.8378  1.0
3  929.116700  1282.5332  1.0
4  930.000000  1196.5275  1.0
1200
            0           1    2
0  906.237038  794.960333  6.0
1  788.753867  811.260433  6.0
2  520.761244  835.494999  7.0
3  373.235013  874.474420  7.0
4  277.067246  894.940203  7.0


# POLICY iteration is applied on data after 1 second time interval

In [81]:
alpha = 0.5
# for j in range(1,2):
#data = pd.read_csv(path+str(j)+"/TouchEvent.csv", header=None)
count = 0
for i in range(len(user_movement_data)-1):
    x = np.digitize(x = [user_movement_data.iloc[i,0]], bins = x_axis)[0]
    y = np.digitize(x = [user_movement_data.iloc[i,1]], bins = y_axis)[0]

    x_next = np.digitize(x = [user_movement_data.iloc[i+1,0]], bins = x_axis)[0]
    y_next = np.digitize(x = [user_movement_data.iloc[i+1,1]], bins = y_axis)[0]

    pressure = user_movement_data.iloc[i,2]
    #contact_size = data.iloc[i,9]
    
    dist = math.sqrt((x-x_next)*(x-x_next)+(y-y_next)*(y-y_next))


    state1 = y*x_cross+x
    state2 = y_next * x_cross+ x_next

    if state1 == state2:
        action = 0
    else:
        action = 1

    TRANSITION_PROBABILITY[state1][action][state2] += 1#0.001#1
    REWARD[state1][action][state2] += dist + pressure#0.001 * pressure#(pressure * contact_size)

    count = count + 1
    if count % 600 == 0: #  # 50 times for 5 seconds, 600 times for 1 minutes(60 seconds)

        for k in range(NO_OF_STATES):
            TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
            TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

        POLICY_iteration()


# POLICY_iteration()
for k in range(NO_OF_STATES):
    TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
    TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

# POLICY_iteration()

print(count)
print('done')

POLICY:  
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.  1.]
 [ 1.  1.  1.  0.  0.  1.]
 [ 1.  0.  1.  0.  1.  0.]
 [ 1.  1.  1.  0.  0.  0.]]
[[  167.44721508   167.44721508   167.44721508   167.44721508
    167.44721508   167.44721508]
 [  167.44721508   167.44721508   167.44721508   167.44721508
    167.44721508   176.17868864]
 [  167.44721508   167.44721508   167.44721508   167.44721508
    167.44721508   167.44721508]
 [  167.44721508   167.44721508   167.44721508   167.44721508
    167.44721508   167.44721508]
 [  167.44721508   309.82186954   218.85066502   167.44721508
    167.44721508   119.23622183]
 [  408.65377796   553.30401705   329.35684696   167.44721508
    167.44721508   102.85474011]
 [  370.92618042  1003.32713486   509.17862666   167.44721508
    320.07052474   167.44721508]
 [  369.71320027   730.96652793   433.89694298   167.44721508
    167.44721508   167.44721508]]
POLICY:  
[[

# testing phase

In [82]:
# error = 0.0
correct_count = 0
acc = []


time_count = 0
for i in range(len(user_movement_data_test)-1):
    x = np.digitize(x = [user_movement_data_test.iloc[i,0]], bins = x_axis)[0]
    y = np.digitize(x = [user_movement_data_test.iloc[i,1]], bins = y_axis)[0]

    x_next = np.digitize(x = [user_movement_data_test.iloc[i+1,0]], bins = x_axis)[0]
    y_next = np.digitize(x = [user_movement_data_test.iloc[i+1,1]], bins = y_axis)[0]

    pressure = user_movement_data_test.iloc[i,2]
    #contact_size = data.iloc[i,9]
    
    dist = math.sqrt((x-x_next)*(x-x_next)+(y-y_next)*(y-y_next))


    state1 = y*x_cross+x
    state2 = y_next * x_cross+ x_next

    if state1 == state2:
        action = 0
    else:
        action = 1
    
    # checking for whether user have taken right action or not
    if POLICY[state1] == action:
        correct_count += 1
        
    #comment
    TRANSITION_PROBABILITY[state1][action][state2] += 1#0.001#1
    REWARD[state1][action][state2] += dist + pressure#0.001 * pressure#(pressure * contact_size)

    time_count = time_count + 1
    if time_count % 300 == 0: #  # 50 times for 5 seconds, 600 times for 1 minutes(60 seconds)

        acc_in = (100*(correct_count))/300.0
        acc.append(acc_in)
        
        correct_count = 0

        # comment
        for k in range(NO_OF_STATES):
            TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
            TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

        POLICY_iteration()


# comment
for k in range(NO_OF_STATES):
    TRANSITION_PROBABILITY[k][0] = TRANSITION_PROBABILITY[k][0]/ np.sum(TRANSITION_PROBABILITY[k][0])
    TRANSITION_PROBABILITY[k][1] = TRANSITION_PROBABILITY[k][1]/ np.sum(TRANSITION_PROBABILITY[k][1])

POLICY_iteration()


print(time_count)


acc_in = (100.0 *(correct_count))/ (time_count%300)
acc.append(acc_in)


POLICY:  
[[ 1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 1.  1.  0.  0.  1.  1.]
 [ 1.  1.  0.  0.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  0.]]
[[  7892.91522057   6729.57933515   6669.59475613   9264.60218229
    7750.12614737   7062.54126423]
 [  6276.00104569   9984.19200699  10406.11301004   9893.94757962
    7786.52587837  11589.41422885]
 [  7359.22538741  10619.07482656   7934.88946628  13923.83718071
    7510.46397656  11276.09012265]
 [ 13785.29958678   9146.26135167  15363.33336667  25550.00003333
   11832.61528992   9128.85371438]
 [  5786.62865355   9329.62434165  12086.6667      23083.33336667
   15873.33336667  10686.23864394]
 [  5277.25003574   5211.47237807   8630.43208689   9655.78090141
    8472.38620203   7559.50912965]
 [  4740.06168972  10287.60322252   6784.23380411   8817.88814837
    6414.69956228   6310.96186557]
 [  5827.44662333   7561.56530671   5174.67778663   6011.81936795


In [83]:
# with modification
print(acc)
print(np.sum(acc)/len(acc))

[58.666666666666664, 63.333333333333336, 57.0, 55.852842809364546]
58.7132107023


In [84]:
# [54.0, 59.333333333333336, 58.333333333333336, 55.666666666666664, 40.666666666666664, 80.0]
# 58.0

# 1
# [17.666666666666668, 65.67164179104478]
# 41.6691542289
# 2
# [33.666666666666664, 52.333333333333336, 78.33333333333333]
# 54.7777777778
# 3
# [75.0]
# 75.0
# 24
# [71.42857142857143]
# 71.4285714286
# 23
# [38.35125448028674]
# 38.3512544803
# 22
# [58.666666666666664, 63.333333333333336, 57.0, 55.852842809364546]
# 58.7132107023